<a href="https://colab.research.google.com/github/icandodoit/Vision/blob/main/sartorious_mmdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install openmim
!mim install mmdet

  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42502 sha256=bf71dd6dd90626d1881024982cc0386c7adb9c66d6cea739c2c7d40ab46d5920
  Stored in directory: /root/.cache/pip/wheels/16/8b/e1/bdebbbc687aa50224a5ce46fe97a040a0c59f92b34bfc750b6
  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8219 sha256=545a91d094e2243eeec3c93449987d51df103de3336f3aa768597d9939a9eb84
  Stored in directory: /root/.cache/pip/wheels/73/2b/f6/26e9f84153c25050fe7c09e88f8e32a6be3c7034a38c418319
Successfully built openmim ordered-set
installing mmdet from https://github.com/open-mmlab/mmdetection.git.
Cloning into '/tmp/tmppxp1jv_q/mmdetection'...
remote: Enumerating objects: 22371, done.
remote: Total 22371 (delta 0), reused 0 (delta 0), pack-reused 22371
Receiving objects: 100% (22371/22371), 25.47 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (15659/15659), done.
Note: checking out '5e246d5e3bc3310b5c625fb57bc03d2338ca39bc'.

You are in 'detached HE

In [1]:
from itertools import groupby
from pycocotools import mask as mutils
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from os import chdir as cd
import os
import pickle
import cv2
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [ ]:
os.mkdir ('/content/datasets/')

In [ ]:
os.mkdir ('/content/datasets/train')

In [ ]:
os.mkdir ('/content/datasets/val')

In [4]:
cd ('/content/sartorius-cell-instance-segmentation/train')
!unzip --qq  /content/drive/MyDrive/kaggle/annotations_train.json.zip

In [5]:
cd ('/content/sartorius-cell-instance-segmentation/train')
!unzip --qq  /content/drive/MyDrive/kaggle/annotations_val.json.zip

In [3]:
#os.mkdir('/content/mmdetection')
cd ('/content')
!unzip --qq /content/drive/MyDrive/kaggle/mm2.19.0.zip

In [2]:
os.mkdir ('/content/sartorius-cell-instance-segmentation')
cd ('/content/sartorius-cell-instance-segmentation')
!unzip --qq /content/drive/MyDrive/kaggle/sartorius-cell-instance-segmentation.zip

In [ ]:
os.mkdir ('/content/datasets/train')

In [13]:
%%writefile /content/mmdetection-2.19.0/configs/_base_/configs.py
#coco seg
dataset_type = 'CocoDataset'
data_root = '/content/sartorius-cell-instance-segmentation/train'
CLASSES = ('cort', 'shsy5y', 'astro')

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='LoadAnnotations', with_bbox=True, with_mask=True, with_seg=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='SegRescale', scale_factor=1 / 8),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + '/train/annotations_train.json',
        img_prefix=data_root + '',
        seg_prefix=data_root + '',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=data_root + '/train/annotations_val.json',
        img_prefix=data_root + '',
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=data_root + '/train/annotations_val.json',
        img_prefix=data_root + '',
        pipeline=test_pipeline))
evaluation = dict(metric=['bbox', 'segm'])



# model settings mask_rcnn_fpn.py
CLASSES = ('cort', 'shsy5y', 'astro')
# model settings
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[4],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=3,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0., 0., 0., 0.],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
        mask_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=3,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
    # model training and testing settings
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            mask_size=28,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100,
            mask_thr_binary=0.5)))
load_from = "https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth"

# optimizer
optimizer = dict(type='SGD', lr=0.0025, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
# learning policy
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=96)



##mmdetection/configs/_base_/default_runtime.py
checkpoint_config = dict(interval=1)
# yapf:disable
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
        # dict(type='TensorboardLoggerHook')
    ])
# yapf:enable
custom_hooks = [dict(type='NumClassCheckHook')]

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]

Overwriting /content/mmdetection-2.19.0/configs/_base_/configs.py


In [14]:
!python /content/mmdetection-2.19.0/tools/train.py /content/mmdetection-2.19.0/configs/_base_/configs.py

2021-12-21 05:49:18,614 - mmdet - INFO - Epoch [8][50/243]	lr: 2.500e-03, eta: 4:41:00, time: 1.104, data_time: 0.650, memory: 8472, loss_rpn_cls: 0.1078, loss_rpn_bbox: 0.1440, loss_cls: 0.4471, acc: 80.5276, loss_bbox: 0.5591, loss_mask: 0.3518, loss: 1.6099
Traceback (most recent call last):
  File "/content/mmdetection-2.19.0/tools/train.py", line 185, in <module>
    main()
  File "/content/mmdetection-2.19.0/tools/train.py", line 181, in main
    meta=meta)
  File "/usr/local/lib/python3.7/dist-packages/mmdet/apis/train.py", line 203, in train_detector
    runner.run(data_loaders, cfg.workflow)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/runner/epoch_based_runner.py", line 127, in run
    epoch_runner(data_loaders[i], **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/runner/epoch_based_runner.py", line 47, in train
    for i, data_batch in enumerate(self.data_loader):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 521, in 

In [15]:
!python /content/mmdetection-2.19.0/tools/test.py /content/mmdetection-2.19.0/configs/_base_/configs.py /content/sartorius-cell-instance-segmentation/train/train/work_dirs/configs/epoch_6.pth --eval bbox segm


loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
load checkpoint from local path: /content/sartorius-cell-instance-segmentation/train/train/work_dirs/configs/epoch_6.pth
[>>] 121/121, 5.7 task/s, elapsed: 21s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=33.01s).
Accumulating evaluation results...
DONE (t=0.19s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.295
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.000
 Average Recall     (AR) @